In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [10]:
# Constants
relevance_score_weight_clicked = 1
relevance_score_weight_booked = 5

In [4]:
df = pd.read_csv("training_set_cleaned.csv")
df_test = pd.read_csv("test_set_cleaned.csv")

In [11]:
# Is the trip international
df['is_international_trip'] = df['prop_country_id'] != df['visitor_location_country_id']
df_test['is_international_trip'] = df_test['prop_country_id'] != df_test['visitor_location_country_id']

# Price difference between property and visitor's mean purchase history
df['price_dif'] = df['price_usd'] - df['visitor_hist_adr_usd']
df_test['price_dif'] = df_test['price_usd'] - df_test['visitor_hist_adr_usd'] 

# Price ratio between property and visitor's mean purchase history
df['price_ratio'] = (df['price_usd']+1)/(df['visitor_hist_adr_usd']+1)
df_test['price_ratio'] = (df_test['price_usd']+1)/(df_test['visitor_hist_adr_usd']+1)

# Star rating difference between property and visitor's mean star rating history
df['star_dif'] = df['prop_starrating'] - df['visitor_hist_starrating']
df_test['star_dif'] = df_test['prop_starrating'] - df_test['visitor_hist_starrating']

# Star rating ratio between property and visitor's mean star rating history
df['star_ratio'] = (df['prop_starrating']+1)/(df['visitor_hist_starrating']+1)
df_test['star_ratio'] = (df_test['prop_starrating']+1)/(df_test['visitor_hist_starrating']+1)

# Ratio of property star rating over property review score
df['star_review_ratio'] = (df['prop_starrating'] + 1)/(df['prop_review_score'] + 1)
df_test['star_review_ratio'] = (df_test['prop_starrating'] + 1)/(df_test['prop_review_score'] + 1)

In [12]:
# Discounted price compared to historical log price
df['discount_score'] = df['prop_log_historical_price'].apply(np.exp) - df['price_usd']
df_test['discount_score'] = df_test['prop_log_historical_price'].apply(np.exp) - df_test['price_usd']


# How popular is the property overall (historically booked or clicked)
prop_click_rate = df.groupby('prop_id')['click_bool'].mean().rename("prop_click_rate")
prop_book_rate = df.groupby('prop_id')['booking_bool'].mean().rename("prop_book_rate")
df = df.merge(prop_click_rate, on='prop_id', how='left')
df = df.merge(prop_book_rate, on='prop_id', how='left')
df_test = df_test.merge(prop_click_rate, on='prop_id', how='left')
df_test = df_test.merge(prop_book_rate, on='prop_id', how='left')

In [13]:
# Average price, star rating, review in a search
avg_price_per_search = df.groupby('srch_id')['price_usd'].mean().rename("average_price_per_search")
avg_star_per_search = df.groupby('srch_id')['prop_starrating'].mean().rename("average_star_per_search")
avg_review_per_search = df.groupby('srch_id')['prop_review_score'].mean().rename("average_review_per_search")
df = df.merge(avg_price_per_search, on='srch_id', how='left')
df = df.merge(avg_star_per_search, on='srch_id', how='left')
df = df.merge(avg_review_per_search, on='srch_id', how='left')

avg_price_per_search_test = df_test.groupby('srch_id')['price_usd'].mean().rename("average_price_per_search")
avg_star_per_search_test = df_test.groupby('srch_id')['prop_starrating'].mean().rename("average_star_per_search")
avg_review_per_search_test = df_test.groupby('srch_id')['prop_review_score'].mean().rename("average_review_per_search")
df_test = df_test.merge(avg_price_per_search_test, on='srch_id', how='left')
df_test = df_test.merge(avg_star_per_search_test, on='srch_id', how='left')
df_test = df_test.merge(avg_review_per_search_test, on='srch_id', how='left')

In [14]:
# Difference between property price and average price per search
df['price_dif2'] = df['price_usd'] - df['average_price_per_search']
df_test['price_dif2'] = df_test['price_usd'] - df_test['average_price_per_search']

df['price_ratio2'] = (df['price_usd']+1)/(df['average_price_per_search']+1)
df_test['price_ratio2'] = (df_test['price_usd']+1)/(df_test['average_price_per_search']+1)

# Difference between property star rating and average star rating per search
df['star_dif2'] = df['prop_starrating'] - df['average_star_per_search']
df_test['star_dif2'] = df_test['prop_starrating'] - df_test['average_star_per_search']

# Ratio between property star rating and average star rating per search
df['star_ratio2'] = (df['prop_starrating']+1)/(df['average_star_per_search']+1)
df_test['star_ratio2'] = (df_test['prop_starrating']+1)/(df_test['average_star_per_search']+1)

# Difference between property review score and average review score per search
df['review_dif'] = df['prop_review_score'] - df['average_review_per_search']
df_test['review_dif'] = df_test['prop_review_score'] - df_test['average_review_per_search']

# Ratio between property review score and average review score per search
df['review_ratio'] = (df['prop_review_score']+1)/(df['average_review_per_search']+1)
df_test['review_ratio'] = (df_test['prop_review_score']+1)/(df_test['average_review_per_search']+1)

In [15]:
avg_position_per_search = df.groupby('prop_id')['position'].mean().rename("average_position_per_property")
df = df.merge(avg_position_per_search, on='prop_id', how='left')
df_test = df_test.merge(avg_position_per_search, on='prop_id', how='left')

In [ ]:
# 1. Identify your numeric columns (exclude prop_id and any others you don't want to aggregate)
num_cols = [
    'price_usd',
    'prop_review_score',
    'srch_query_affinity_score',
    'visitor_hist_starrating',
    'visitor_hist_adr_usd',
    'prop_starrating',
    'prop_location_score1',
    'prop_location_score2',
    'prop_log_historical_price',
    'price_usd',
    'srch_length_of_stay',
    'srch_booking_window',
    'srch_adults_count',
    'srch_children_count',
    'srch_room_count',
    'srch_saturday_night_bool',
    'srch_query_affinity_score',
    'orig_destination_distance',
    'random_bool'
]
# Result has prop_id 101, 102

# 2. Compute the group-by aggregations
agg = (
    df
    .groupby('prop_id')[num_cols]
    .agg(['mean', 'std', 'median'])
)


# 3. Flatten the multi-index columns
agg.columns = [
    f"{col}_{stat}"
    for col, stat in agg.columns
]
agg = agg.reset_index()

# 4. Merge back onto your original DataFrame
df = df.merge(agg, on='prop_id', how='left')
df_test = df_test.merge(agg, on='prop_id', how='left')

In [21]:
# 1. Make sure it's a datetime type
df['date_time'] = pd.to_datetime(df['date_time'])
df_test['date_time'] = pd.to_datetime(df_test['date_time'])

# 2. Extract the month (as an integer 1–12)
df['month'] = df['date_time'].dt.month
df_test['month'] = df_test['date_time'].dt.month

df.drop('date_time', axis=1, inplace=True, errors='ignore')
df_test.drop('date_time', axis=1, inplace=True, errors='ignore')

In [22]:
df['relevance_score'] = relevance_score_weight_clicked * df['click_bool'] + relevance_score_weight_booked * df['booking_bool']

In [23]:
df.fillna(df.median(), inplace=True)
df_test.fillna(df.median(), inplace=True)

print(df.isnull().sum()[df.isnull().sum() > 0])
print(df_test.isnull().sum()[df_test.isnull().sum() > 0])

Series([], dtype: int64)
Series([], dtype: int64)


In [24]:
# Save to a new file
df.to_csv("training_set_final.csv", index=False)
df_test.to_csv("test_set_final.csv", index=False)